In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
import re
import json
# from sklearn import cross_validation as cv
from collections import Counter
# from tensorflow.python.ops import math_ops
import matplotlib.pyplot as plt
import time
import datetime

In [2]:
ani_ali = pd.read_csv('./911call/chifoia_event_ani_ali.csv')

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ani_ali.head()

,Unnamed: 0,ALINUMBER,ASQDATE,ANSWERDATE,POSITION,PERSID,WORKSTATION,EQUIPMENT,DESCR,ACDID,COMPANYID,XCOORD,YCOORD,DIRECTION
0,0,89375091,01/01/2016 00:00:04,01/01/2016 00:00:00,732.0,C549511,PD32,WPH2,NaN,NaN,NaN,NaN,NaN,NaN
1,1,89375092,01/01/2016 00:00:09,01/01/2016 00:00:00,725.0,C606055,PCT25,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,89375093,01/01/2016 00:00:13,01/01/2016 00:00:00,748.0,C100965,PCT48,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
3,3,89375094,01/01/2016 00:00:18,01/01/2016 00:00:00,749.0,C100963,PCT49,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
4,4,89375095,01/01/2016 00:00:19,01/01/2016 00:00:00,721.0,A99592,PADM21,CNTX,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
ani_ali.ALINUMBER.unique().shape

(6751903,)

In [5]:
ani_ali.ASQDATE.unique()

array(['01/01/2016 00:00:04', '01/01/2016 00:00:09', '01/01/2016 00:00:13',
       ..., '12/31/2016 23:59:24', '12/31/2016 23:59:38',
       '12/31/2016 23:59:51'], dtype=object)

In [6]:
ani_ali.ANSWERDATE.unique()

array(['01/01/2016 00:00:00', nan, '01/01/2016 00:01:00', ...,
       '12/31/2016 23:57:00', '12/31/2016 23:58:00', '12/31/2016 23:59:00'], dtype=object)

In [7]:
ani_ali.XCOORD.unique().shape

(34132,)

In [8]:
ani_ali = ani_ali.drop(columns=['Unnamed: 0'])

In [9]:
ani_ali.ASQDATE = pd.to_datetime(ani_ali.ASQDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')

In [10]:
ani_ali.ANSWERDATE = pd.to_datetime(ani_ali.ANSWERDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')

In [11]:
ani_ali

,ALINUMBER,ASQDATE,ANSWERDATE,POSITION,PERSID,WORKSTATION,EQUIPMENT,DESCR,ACDID,COMPANYID,XCOORD,YCOORD,DIRECTION
0,89375091,2016-01-01 00:00:04,2016-01-01 00:00:00,732.0,C549511,PD32,WPH2,NaN,NaN,NaN,NaN,NaN,NaN
1,89375092,2016-01-01 00:00:09,2016-01-01 00:00:00,725.0,C606055,PCT25,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
2,89375093,2016-01-01 00:00:13,2016-01-01 00:00:00,748.0,C100965,PCT48,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
3,89375094,2016-01-01 00:00:18,2016-01-01 00:00:00,749.0,C100963,PCT49,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
4,89375095,2016-01-01 00:00:19,2016-01-01 00:00:00,721.0,A99592,PADM21,CNTX,NaN,NaN,NaN,NaN,NaN,NaN
5,89375096,2016-01-01 00:00:20,2016-01-01 00:00:00,738.0,C113107,PCT38,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
6,89375097,2016-01-01 00:00:24,2016-01-01 00:00:00,834.0,C839505,PD124,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
7,89375098,2016-01-01 00:00:25,2016-01-01 00:00:00,711.0,C307554,PCT11,WPH1,NaN,NaN,NaN,NaN,NaN,NaN
8,89375099,2016-01-01 00:00:28,NaT,708.0,A560946,PD08,NaN,RECORD NOT FOUND,NaN,NaN,NaN,NaN,NaN
9,89375100,2016-01-01 00:00:30,2016-01-01 00:00:00,793.0,C108849,PDT93,WPH1,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
event_case = pd.read_csv('./911call/chifoia_event_case.csv')

In [13]:
event_case.head()

,Unnamed: 0,CASETYPE,EVENTNUMBER,UNIT,CASENUMBER,ENTRYDATE,ENTRYDATEHN,CASECANFLAG
0,0,RDN,1535016063,5151,HZ105113,01/05/2016 16:49:43,8,N
1,1,RDN,1536514126,1723R,HZ100004,01/01/2016 00:05:45,30,N
2,2,RDN,1536514231,934R,HZ100020,01/01/2016 00:43:06,16,N
3,3,RDN,1536514255,911R,HZ100003,01/01/2016 00:03:07,73,N
4,4,RDN,1536514304,2523R,HZ100001,01/01/2016 00:00:10,31,N


In [14]:
event_case = event_case.drop(columns=['Unnamed: 0'])

In [15]:
event_case

,CASETYPE,EVENTNUMBER,UNIT,CASENUMBER,ENTRYDATE,ENTRYDATEHN,CASECANFLAG
0,RDN,1535016063,5151,HZ105113,01/05/2016 16:49:43,8,N
1,RDN,1536514126,1723R,HZ100004,01/01/2016 00:05:45,30,N
2,RDN,1536514231,934R,HZ100020,01/01/2016 00:43:06,16,N
3,RDN,1536514255,911R,HZ100003,01/01/2016 00:03:07,73,N
4,RDN,1536514304,2523R,HZ100001,01/01/2016 00:00:10,31,N
5,RDN,1536514384,322,HZ100104,01/01/2016 02:15:27,15,N
6,RDN,1536514600,1915R,HZ100002,01/01/2016 00:00:43,74,N
7,RDN,1536514703,323,HZ100014,01/01/2016 00:32:57,5,N
8,RDN,1600100021,2411R,HZ100010,01/01/2016 00:23:28,38,N
9,RDN,1600100059,1561D,HZ100007,01/01/2016 00:18:41,99,N


In [16]:
event_case.ENTRYDATE = pd.to_datetime(event_case.ENTRYDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')

In [17]:
event_case

,CASETYPE,EVENTNUMBER,UNIT,CASENUMBER,ENTRYDATE,ENTRYDATEHN,CASECANFLAG
0,RDN,1535016063,5151,HZ105113,2016-01-05 16:49:43,8,N
1,RDN,1536514126,1723R,HZ100004,2016-01-01 00:05:45,30,N
2,RDN,1536514231,934R,HZ100020,2016-01-01 00:43:06,16,N
3,RDN,1536514255,911R,HZ100003,2016-01-01 00:03:07,73,N
4,RDN,1536514304,2523R,HZ100001,2016-01-01 00:00:10,31,N
5,RDN,1536514384,322,HZ100104,2016-01-01 02:15:27,15,N
6,RDN,1536514600,1915R,HZ100002,2016-01-01 00:00:43,74,N
7,RDN,1536514703,323,HZ100014,2016-01-01 00:32:57,5,N
8,RDN,1600100021,2411R,HZ100010,2016-01-01 00:23:28,38,N
9,RDN,1600100059,1561D,HZ100007,2016-01-01 00:18:41,99,N


In [18]:
event_case.ENTRYDATE.max()

Timestamp('2017-08-31 23:59:40')

In [19]:
cross_reference = pd.read_csv('./911call/chifoia_event_cross_reference.csv')

In [20]:
cross_reference.head()

,Unnamed: 0,EVENTNUMBER,CROSSEVENTNUMBER,CROSSTYPEFLAG
0,0,1600100014,1600100162,DUP
1,1,1600100014,1600100436,DUP
2,2,1600100015,1536514749,DUP
3,3,1600100015,160010002,911
4,4,1600100017,160010003,911


In [22]:
cross_reference = cross_reference.drop(columns=['Unnamed: 0'])

In [23]:
cross_reference

,EVENTNUMBER,CROSSEVENTNUMBER,CROSSTYPEFLAG
0,1600100014,1600100162,DUP
1,1600100014,1600100436,DUP
2,1600100015,1536514749,DUP
3,1600100015,160010002,911
4,1600100017,160010003,911
5,1600100018,1600100062,DUP
6,1600100027,1536514603,DUP
7,1600100034,1600100045,DUP
8,1600100034,1600100053,DUP
9,1600100040,1536514630,DUP


In [24]:
cross_reference.CROSSTYPEFLAG.unique()

array(['DUP', '911', 'CPY', nan, 'OPR', 'TWO'], dtype=object)

1 ‐‐ Duplicate

2 ‐‐ Operator

3 ‐‐ Combined Event

4 ‐‐ Copied Event

5 ‐‐ 911 Cross Reference

6 ‐‐ Events Generated Instead of Duplicate

8 ‐‐ Police to Streets & Sanitation Event

9 ‐‐ Streets & Sanitation Event to Police

That is the description in the pdf file

In [25]:
event_disposition = pd.read_csv('./911call/chifoia_event_disposition.csv')

In [26]:
event_disposition.head()

,Unnamed: 0,EVENTNUMBER,UNIT,DISPOSITION,OCCCNT
0,0,1600100001,2525R,1P,1
1,1,1600100003,NaN,FILE,1
2,2,1600100004,NaN,NaN,1
3,3,1600100005,2433R,19P,1
4,4,1600100006,NaN,FILE,1


In [27]:
event_disposition = event_disposition.drop(columns = ['Unnamed: 0'])

In [28]:
event_disposition

,EVENTNUMBER,UNIT,DISPOSITION,OCCCNT
0,1600100001,2525R,1P,1
1,1600100003,NaN,FILE,1
2,1600100004,NaN,NaN,1
3,1600100005,2433R,19P,1
4,1600100006,NaN,FILE,1
5,1600100007,722R,19P,1
6,1600100008,NaN,NaN,1
7,1600100009,NaN,FILE,1
8,1600100010,NaN,FILE,1
9,1600100011,2561,19P,1


In [29]:
event_geo = pd.read_csv('./911call/chifoia_event_geo.csv')

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
event_geo.head()

,Unnamed: 0,EVENTNUMBER,TYPE,ATOMPOL,WARD,ZIPCODE,SEGMENTID,INTERSECTID,DISPGROUP,RESPAREA,XCOORD,YCOORD
0,0,1600100001,O,2525.0,NaN,NaN,223938,0,25,2525.0,-87729026.0,41918031.0
1,1,1600100001,S,2525.0,35.0,60639.0,223938,0,25,2525.0,-87729026.0,41918031.0
2,2,1600100002,O,1224.0,NaN,NaN,236608,0,NaN,NaN,-87662528.0,41881528.0
3,3,1600100002,S,1224.0,27.0,60607.0,236608,0,12,1224.0,-87662528.0,41881528.0
4,4,1600100004,O,1215.0,NaN,NaN,236622,0,12,1215.0,-87662682.0,41885411.0


In [32]:
event_geo.loc[event_geo.XCOORD.abs()<100]

,Unnamed: 0,EVENTNUMBER,TYPE,ATOMPOL,WARD,ZIPCODE,SEGMENTID,INTERSECTID,DISPGROUP,RESPAREA,XCOORD,YCOORD
115001,115001,1600514109,S,2031.0,47.0,60625.0,0,82788,20,2031.0,0.0,0.0
186533,186533,1600817308,O,122.0,NaN,NaN,0,72994,1,122.0,0.0,0.0
186534,186534,1600817308,S,122.0,42.0,60606.0,0,72994,1,122.0,0.0,0.0
226754,226754,1601010206,O,1923.0,NaN,NaN,0,80417,19,1923.0,0.0,0.0
226755,226755,1601010206,S,1923.0,44.0,60613.0,0,80417,19,1923.0,0.0,0.0
348379,348379,1601600356,O,114.0,NaN,NaN,0,81642,1,114.0,0.0,0.0
348380,348380,1601600356,S,114.0,42.0,60601.0,0,81642,1,114.0,0.0,0.0
358428,358428,1601607134,O,114.0,NaN,NaN,0,83395,1,114.0,0.0,0.0
358429,358429,1601607134,S,114.0,42.0,60601.0,0,83395,1,114.0,0.0,0.0
375970,375970,1601702550,O,111.0,NaN,NaN,0,73075,1,111.0,0.0,0.0


The XCORD and YCORD is 1E6 times of the lat and lon.

In [33]:
event_geo = event_geo.drop(columns =['Unnamed: 0'])

In [34]:
event_geo.head()

,EVENTNUMBER,TYPE,ATOMPOL,WARD,ZIPCODE,SEGMENTID,INTERSECTID,DISPGROUP,RESPAREA,XCOORD,YCOORD
0,1600100001,O,2525.0,NaN,NaN,223938,0,25,2525.0,-87729026.0,41918031.0
1,1600100001,S,2525.0,35.0,60639.0,223938,0,25,2525.0,-87729026.0,41918031.0
2,1600100002,O,1224.0,NaN,NaN,236608,0,NaN,NaN,-87662528.0,41881528.0
3,1600100002,S,1224.0,27.0,60607.0,236608,0,12,1224.0,-87662528.0,41881528.0
4,1600100004,O,1215.0,NaN,NaN,236622,0,12,1215.0,-87662682.0,41885411.0


In [35]:
event_geo.XCOORD = event_geo.XCOORD/1000000
event_geo.YCOORD = event_geo.YCOORD/1000000

In [36]:
event_geo.head()

,EVENTNUMBER,TYPE,ATOMPOL,WARD,ZIPCODE,SEGMENTID,INTERSECTID,DISPGROUP,RESPAREA,XCOORD,YCOORD
0,1600100001,O,2525.0,NaN,NaN,223938,0,25,2525.0,-87.729026,41.918031
1,1600100001,S,2525.0,35.0,60639.0,223938,0,25,2525.0,-87.729026,41.918031
2,1600100002,O,1224.0,NaN,NaN,236608,0,NaN,NaN,-87.662528,41.881528
3,1600100002,S,1224.0,27.0,60607.0,236608,0,12,1224.0,-87.662528,41.881528
4,1600100004,O,1215.0,NaN,NaN,236622,0,12,1215.0,-87.662682,41.885411


In [2]:
event_history_header = pd.read_csv('./911call/chifoia_event_history_header.csv')

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,26,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
event_history_header.head()

,Unnamed: 0,EVENTNUMBER,RECDDATE,RECDDATEHN,ENTRYDATE,ENTRYDATEHN,CLOSEDATE,CLOSEDATEHN,STATUS,ENTRYPERSID,...,ACKDATE,ENRDATE,ONSDATE,CANCELEDFLAG,ADDROFOCCOVERRIDEFLAG,DISPATCHED_FLAG,DUPLICATE_FLAG,FILE_ADVISED_FLAG,CASE_NUMBER_FLAG,WIRELESS_FLAG
0,0,1600100001,12/31/2015 23:58:16,71,01/01/2016 00:00:01,8,01/01/2016 00:39:10,78,CLS,C307502,...,01/01/2016 00:30:30,01/01/2016 00:34:34,01/01/2016 00:35:44,N,N,N,N,N,N,N
1,1,1600100002,NaN,0,01/01/2016 00:00:02,74,01/01/2016 21:01:15,46,CLS,D106922,...,NaN,NaN,NaN,N,N,N,N,Y,N,N
2,2,1600100003,12/31/2015 23:57:37,44,01/01/2016 00:00:10,20,01/01/2016 00:00:10,20,CLS,C606057,...,NaN,NaN,NaN,N,Y,N,N,Y,N,N
3,3,1600100004,12/31/2015 23:59:09,42,01/01/2016 00:00:12,97,01/01/2016 00:00:12,97,CLS,C100957,...,NaN,NaN,NaN,N,N,N,N,Y,N,N
4,4,1600100005,12/31/2015 23:59:25,97,01/01/2016 00:00:16,69,01/01/2016 00:01:05,29,CLS,C602883,...,NaN,NaN,NaN,N,N,N,N,N,N,N


In [4]:
event_history_header = event_history_header.drop(columns =['Unnamed: 0'])

In [6]:
event_history_header.RECDDATE = pd.to_datetime(event_history_header.RECDDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')
event_history_header.ENTRYDATE = pd.to_datetime(event_history_header.ENTRYDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')
event_history_header.CLOSEDATE = pd.to_datetime(event_history_header.CLOSEDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')
event_history_header.ACKDATE = pd.to_datetime(event_history_header.ACKDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')
event_history_header.ENRDATE = pd.to_datetime(event_history_header.ENRDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')
event_history_header.ONSDATE = pd.to_datetime(event_history_header.ONSDATE,format = '%m/%d/%Y %H:%M:%S', errors='coerce')

In [7]:
event_history_header

,EVENTNUMBER,RECDDATE,RECDDATEHN,ENTRYDATE,ENTRYDATEHN,CLOSEDATE,CLOSEDATEHN,STATUS,ENTRYPERSID,ENTRYWKS,...,ACKDATE,ENRDATE,ONSDATE,CANCELEDFLAG,ADDROFOCCOVERRIDEFLAG,DISPATCHED_FLAG,DUPLICATE_FLAG,FILE_ADVISED_FLAG,CASE_NUMBER_FLAG,WIRELESS_FLAG
0,1600100001,2015-12-31 23:58:16,71,2016-01-01 00:00:01,8,2016-01-01 00:39:10,78,CLS,C307502,PD53,...,2016-01-01 00:30:30,2016-01-01 00:34:34,2016-01-01 00:35:44,N,N,N,N,N,N,N
1,1600100002,NaT,0,2016-01-01 00:00:02,74,2016-01-01 21:01:15,46,CLS,D106922,PDT96,...,NaT,NaT,NaT,N,N,N,N,Y,N,N
2,1600100003,2015-12-31 23:57:37,44,2016-01-01 00:00:10,20,2016-01-01 00:00:10,20,CLS,C606057,PCT59,...,NaT,NaT,NaT,N,Y,N,N,Y,N,N
3,1600100004,2015-12-31 23:59:09,42,2016-01-01 00:00:12,97,2016-01-01 00:00:12,97,CLS,C100957,PCT14,...,NaT,NaT,NaT,N,N,N,N,Y,N,N
4,1600100005,2015-12-31 23:59:25,97,2016-01-01 00:00:16,69,2016-01-01 00:01:05,29,CLS,C602883,PCT23,...,NaT,NaT,NaT,N,N,N,N,N,N,N
5,1600100006,2015-12-31 23:59:56,59,2016-01-01 00:00:17,36,2016-01-01 00:00:17,36,CLS,C8782,PCT39,...,NaT,NaT,NaT,N,N,N,N,Y,N,N
6,1600100007,2015-12-31 23:59:12,5,2016-01-01 00:00:25,35,2016-01-01 00:39:43,35,CLS,C99440,PCT54,...,2016-01-01 00:01:15,2016-01-01 00:00:45,NaT,N,N,N,N,N,N,N
7,1600100008,2016-01-01 00:00:20,59,2016-01-01 00:00:27,50,2016-01-01 00:00:27,44,CLS,C113107,PCT38,...,NaT,NaT,NaT,N,N,N,N,Y,N,N
8,1600100009,2016-01-01 00:00:09,0,2016-01-01 00:00:28,8,2016-01-01 00:00:28,7,CLS,C606055,PCT25,...,NaT,NaT,NaT,N,N,N,N,Y,N,N
9,1600100010,2015-12-31 23:57:26,94,2016-01-01 00:00:28,84,2016-01-01 00:00:28,84,CLS,C307554,PCT11,...,NaT,NaT,NaT,N,N,N,N,Y,N,N


In [8]:
event_history_header.ACKDATE.max()

Timestamp('2017-09-01 20:21:23')

In [2]:
event_personnel_link = pd.read_csv('./911call/chifoia_event_personnel_link.csv')

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
event_personnel_link.head()

,Unnamed: 0,EVENTNUMBER,UNIT,UNAOCCCNT,PERSID,UNITLOGONDATE,UNITLOGOFFDATE
0,0,1600100001,2525R,1,17029,NaN,NaN
1,1,1600100001,2525R,1,3325,NaN,NaN
2,2,1600100001,2535R,2,18494,NaN,NaN
3,3,1600100001,2535R,2,9427,NaN,NaN
4,4,1600100005,2433R,1,10542,NaN,NaN


In [5]:
event_personnel_link = event_personnel_link.drop(columns=['Unnamed: 0'])

In [6]:
event_personnel_link

,EVENTNUMBER,UNIT,UNAOCCCNT,PERSID,UNITLOGONDATE,UNITLOGOFFDATE
0,1600100001,2525R,1,17029,NaN,NaN
1,1600100001,2525R,1,3325,NaN,NaN
2,1600100001,2535R,2,18494,NaN,NaN
3,1600100001,2535R,2,9427,NaN,NaN
4,1600100005,2433R,1,10542,NaN,NaN
5,1600100005,2433R,1,7476,NaN,NaN
6,1600100007,722R,1,17155,NaN,NaN
7,1600100007,722R,1,19221,NaN,NaN
8,1600100014,822R,2,18876,NaN,NaN
9,1600100014,822R,2,8526,NaN,NaN


In [7]:
event_segment = pd.read_csv('./911call/chifoia_event_segment.csv')

In [8]:
event_segment.head()

,Unnamed: 0,EVENTNUMBER,OCCCNT,CREATIONDATE,CREATIONDATEHN,OVERRIDEDATE,OVERRIDEDATEHN,SEGMENTNAME,SEGMENTDATE,SEGMENTDATEHN,CREATIONFLAG,SOURCEFLAG,WORKSTATION,PERSID,UNIT,UNITLOGONDATE
0,0,1600100001,0,12/31/2015 23:58:16,NaN,NaN,NaN,REC,12/31/2015 23:58:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1600100001,10001,NaN,NaN,NaN,NaN,RMKS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1600100001,10002,NaN,NaN,NaN,NaN,RMKS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1600100001,10003,NaN,NaN,NaN,NaN,RMKS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1600100001,10004,NaN,NaN,NaN,NaN,RMKS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
event_segment = event_segment.drop(columns=['Unnamed: 0'])

In [ ]:
## Date time convert later

In [2]:
event_unit = pd.read_csv('./911call/chifoia_event_unit.csv')

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
event_unit.head()

,Unnamed: 0,EVENTNUMBER,UNIT,UNAOCCCNT,OCCCNT,STATUSNAME,CREATIONDATE,OVERRIDEDATE,STACKEDFLAG
0,0,1600100001,2525R,1,1,DIS,01/01/2016 00:30:17,NaN,NaN
1,1,1600100001,2525R,1,2,ACK,01/01/2016 00:30:30,NaN,NaN
2,2,1600100001,2525R,1,3,ENR,01/01/2016 00:35:42,NaN,NaN
3,3,1600100001,2525R,1,4,ONS,01/01/2016 00:35:44,NaN,NaN
4,4,1600100001,2525R,1,5,AV,01/01/2016 00:39:10,NaN,NaN


In [5]:
event_unit = event_unit.drop(columns=['Unnamed: 0'])

In [6]:
event_unit.head()

,EVENTNUMBER,UNIT,UNAOCCCNT,OCCCNT,STATUSNAME,CREATIONDATE,OVERRIDEDATE,STACKEDFLAG
0,1600100001,2525R,1,1,DIS,01/01/2016 00:30:17,NaN,NaN
1,1600100001,2525R,1,2,ACK,01/01/2016 00:30:30,NaN,NaN
2,1600100001,2525R,1,3,ENR,01/01/2016 00:35:42,NaN,NaN
3,1600100001,2525R,1,4,ONS,01/01/2016 00:35:44,NaN,NaN
4,1600100001,2525R,1,5,AV,01/01/2016 00:39:10,NaN,NaN


In [8]:
unit_history_header = pd.read_csv('./911call/chifoia_unit_history_header.csv')

In [9]:
unit_history_header.head()

,Unnamed: 0,UNIT,DISPGROUP,UNITLOGONDATE,UNITLOGONDATEHN,UNITLOGOFFDATE,UNITLOGOFFDATEHN,SEGOCCCNTHI,RESPAREA
0,0,1000X,010,01/08/2016 16:10:30,10,01/09/2016 01:20:22,10,3,NaN
1,1,1000X,010,01/14/2016 09:36:24,74,01/14/2016 17:58:17,6,5,NaN
2,2,1000X,010,01/15/2016 15:49:41,35,01/16/2016 00:46:30,4,11,NaN
3,3,1000X,010,01/21/2016 12:09:20,77,01/21/2016 15:46:39,95,4,NaN
4,4,1000X,010,01/22/2016 16:07:52,18,01/23/2016 01:17:45,93,8,NaN


In [10]:
unit_history_header = unit_history_header.drop(columns=['Unnamed: 0'])

In [11]:
unit_history_header.head()

,UNIT,DISPGROUP,UNITLOGONDATE,UNITLOGONDATEHN,UNITLOGOFFDATE,UNITLOGOFFDATEHN,SEGOCCCNTHI,RESPAREA
0,1000X,010,01/08/2016 16:10:30,10,01/09/2016 01:20:22,10,3,NaN
1,1000X,010,01/14/2016 09:36:24,74,01/14/2016 17:58:17,6,5,NaN
2,1000X,010,01/15/2016 15:49:41,35,01/16/2016 00:46:30,4,11,NaN
3,1000X,010,01/21/2016 12:09:20,77,01/21/2016 15:46:39,95,4,NaN
4,1000X,010,01/22/2016 16:07:52,18,01/23/2016 01:17:45,93,8,NaN


In [12]:
unit_personnel_link = pd.read_csv('./911call/chifoia_unit_personnel_link.csv')

In [13]:
unit_personnel_link.head()

,Unnamed: 0,PERSID,UNIT,UNITLOGONDATE,UNITLOGOFFDATE
0,0,10000,706DR,01/20/2016 21:53:22,01/20/2016 22:18:46
1,1,10000,706DR,01/21/2016 21:36:35,01/22/2016 03:42:36
2,2,10000,706DR,01/25/2016 21:40:55,01/26/2016 05:18:39
3,3,10000,706DR,01/26/2016 21:37:56,01/26/2016 22:01:49
4,4,10000,706DR,01/26/2016 23:05:20,01/27/2016 05:37:23


In [14]:
unit_segment = pd.read_csv('./911call/chifoia_unit_segment.csv')

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
unit_segment.head()

,Unnamed: 0,UNIT,OCCCNT,UNITLOGONDATE,CREATIONDATE,CREATIONDATEHN,OVERRIDEDATE,OVERRIDEDATEHN,SEGMENTNAME,SEGMENTDATE,SEGMENTDATEHN,WORKSTATION,PERSID,EVENTNUMBER,CREATIONFLAG,SOURCEFLAG
0,0,1000X,10,01/15/2016 15:49:41,01/15/2016 23:21:29,54,NaN,0,REMINQ,01/15/2016 23:21:29,54,PMDT2404,PC0G213,,O,NaN
1,1,1000X,11,01/15/2016 15:49:41,01/16/2016 00:46:30,4,NaN,0,LOGOFF,01/16/2016 00:46:30,4,PMDT2404,PC0G213,NaN,O,NaN
2,2,1000X,1,01/08/2016 16:10:30,01/08/2016 16:10:30,10,NaN,0,LOGON,01/08/2016 16:10:30,10,PMDT2404,PC0G213,NaN,O,NaN
3,3,1000X,1,01/14/2016 09:36:24,01/14/2016 09:36:24,74,NaN,0,LOGON,01/14/2016 09:36:24,74,PMDT2404,PC0G213,NaN,O,NaN
4,4,1000X,1,01/15/2016 15:49:41,01/15/2016 15:49:41,35,NaN,0,LOGON,01/15/2016 15:49:41,35,PMDT2404,PC0G213,NaN,O,NaN


In [ ]:
## There might be some problems in the csv files. Next time read txt file directly!!!